<h1 align="center"><font color="yellow">Aproveitando a geração aumentada de retrieval (recuperação) com LangChain</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

# Contextualizando

A `Retrieval Augmented Generation` (Geração Aumentada de Recuperação - RAG) é mais do que apenas uma palavra da moda na comunidade de desenvolvedores de `IA`; é uma abordagem inovadora que está ganhando força rapidamente em organizações e empresas de todos os tamanhos.

À medida que nos aprofundamos nos recursos dos Large Language Models (`LLMs`), descobrindo novos aplicativos ao longo do caminho, o valor e o apelo do `RAG` estão se tornando cada vez mais claros. E por um bom motivo!

Apesar de seu recente aumento de popularidade, as bases da `Geração Aumentada de Recuperação` foram lançadas em `2020` quando o `Facebook AI Research` (`FAIR`) popularizou essa abordagem inovadora em seu artigo seminal, [“Geração Aumentada de Recuperação para Tarefas NLP Intensivas em Conhecimento”](https://arxiv.org/abs/2005.11401).


Desde então, essa mudança de paradigma catalisou avanços significativos no Processamento de Linguagem Natural (NLP), oferecendo uma metodologia exclusiva para lidar com tarefas intensivas em conhecimento e específicas de domínio com `LLMs`.

# Geração Aumentada

Para ter uma noção de como o `RAG` funciona, vamos primeiro dar uma olhada na Geração Aumentada, pois ela sustenta a abordagem.

A geração aumentada significa simplesmente adicionar informações externas ao `prompt` de entrada alimentado no `LLM`, **aumentando assim a resposta gerada**. Um exemplo simples de uso de um prompt de contexto aumentado com `Langchain` é o seguinte:

`load_qa_chain`

<font color="orange">Carrega uma cadeia que você pode usar para fazer `QA` em um conjunto de documentos, mas usa TODOS esses documentos.

`chain_type="stuff"` não funcionará porque o número de tokens excede o limite. Podemos tentar outros tipos de cadeia como `"map_reduce"`.</font>

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# texto = """Um fenótipo refere-se às propriedades físicas observáveis \
#                     de um organismo, incluindo sua aparência, desenvolvimento \
#                     e comportamento. É determinado tanto pelo genótipo do organismo, \
#                     que é o conjunto de genes que ele carrega, quanto pelas influências \
#                     ambientais sobre esses genes."""

# Carregar o documento de uma string:
loader = TextLoader("texto.txt")
docs = loader.load()

# Crie o Template de prompt para base qa_chain:
qa_template = """As informações de contexto estão abaixo.
    ---------------------
    {context}
    ---------------------
    Dada a informação do contexto e não o conhecimento prévio, 
    Responda à pergunta: {question}
    Resposta:
"""

PROMPT = PromptTemplate(template=qa_template,
                        input_variables=["context", "question"]
                       )

chain = load_qa_chain(OpenAI(temperature=0),
                      chain_type="stuff",
                      prompt=PROMPT
                     )

query = "O que é um fenótipo?"

chain({"input_documents": docs, "question": query}, return_only_outputs=False)


{'input_documents': [Document(page_content='Um fenótipo refere-se às propriedades físicas observáveis \\\n                    de um organismo, incluindo sua aparência, desenvolvimento \\\n                    e comportamento. É determinado tanto pelo genótipo do organismo, \\\n                    que é o conjunto de genes que ele carrega, quanto pelas influências \\\n                    ambientais sobre esses genes.', metadata={'source': 'texto.txt'})],
 'question': 'O que é um fenótipo?',
 'output_text': '    Um fenótipo é a manifestação externa de um organismo, que inclui sua aparência, desenvolvimento e comportamento. É determinado pelo genótipo do organismo, que é o conjunto de genes que ele carrega, bem como pelas influências ambientais sobre esses genes.'}

<font color="orange">Aqui, inserimos algum contexto adicional no `prompt`, antes de pedir ao modelo que responda à nossa pergunta.

Ao projetar o prompt para o LLM dessa maneira - uma instrução seguida por `{context}` e, em seguida, `{question}`, podemos orientar o modelo a aderir ao contexto específico que fornecemos para responder à pergunta. Isso é facilmente alcançado usando a cadeia `stuff` no `QAChain` básico do Langchain.</font>

In [3]:
from langchain.document_loaders import WebBaseLoader

urls = ["https://platform.openai.com/docs/guides/gpt-best-practices/",
"https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
"https://github.com/brexhq/prompt-engineering"]

loader = WebBaseLoader(urls)
data = loader.load()

data

[Document(page_content='OpenAI PlatformYou need to enable JavaScript to run this app.', metadata={'source': 'https://platform.openai.com/docs/guides/gpt-best-practices/', 'title': 'OpenAI Platform', 'description': "Explore resources, tutorials, API docs, and dynamic examples to get the most out of OpenAI's developer platform.", 'language': 'en'}),
 Document(page_content='\n\n\n\n\n\nPrompt Engineering | Lil\'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil\'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      Prompt Engineering\n    \nMarch 15, 2023\xa0·\xa021 min\xa0·\xa0Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nBasic Prompting\n\nZero-Shot\n\nFew-shot\n\nTips for Example Selection\n\nTips for Example Ordering\n\n\n\nInstruction Prompting\n\nSelf-Consistency Sampling\n\nChain-of-Thought (CoT)\n\nTypes of CoT prompts\n\nTips and Ex

In [4]:
import tiktoken

encoding_name = tiktoken.get_encoding("cl100k_base")

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Retorna o número de tokens numa string de texto."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


In [5]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=25)

docs = text_splitter.split_documents(data)

In [6]:
docs 

[Document(page_content='OpenAI PlatformYou need to enable JavaScript to run this app.', metadata={'source': 'https://platform.openai.com/docs/guides/gpt-best-practices/', 'title': 'OpenAI Platform', 'description': "Explore resources, tutorials, API docs, and dynamic examples to get the most out of OpenAI's developer platform.", 'language': 'en'}),
 Document(page_content="\n\n\n\n\n\nPrompt Engineering | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      Prompt Engineering\n    \nMarch 15, 2023\xa0·\xa021 min\xa0·\xa0Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nBasic Prompting\n\nZero-Shot\n\nFew-shot\n\nTips for Example Selection\n\nTips for Example Ordering\n\n\n\nInstruction Prompting\n\nSelf-Consistency Sampling\n\nChain-of-Thought (CoT)\n\nTypes of CoT prompts\n\nTips and Exte